In [1]:
import pandas as pd
import numpy as np

In [111]:
medalists = pd.read_excel('medalists.xlsx', sheetname=2, skiprows=[0,1,2,3])
medalists.head()


,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [116]:
countryCodes = pd.read_excel('medalists.xlsx', sheetname=5, header=0, parse_cols=2)
countryCodes = countryCodes.rename(columns={'Int Olympic Committee code': 'IOC', 'ISO code': 'ISO'})
countryCodes.head()

,Country,IOC,ISO
0,Afghanistan,AFG,AF
1,Albania,ALB,AL
2,Algeria,ALG,DZ
3,American Samoa*,ASA,AS
4,Andorra,AND,AD


Transform the data so each olympics is a row, each country is a column, and each cell is the number of medals won by a country at that particular olympics

In [85]:
byCountry = medalists.groupby(['Edition', 'NOC']).size().unstack()
byCountry.index.name = byCountry.columns.name = None
byCountry.head()

,AFG,AHO,ALG,ANZ,ARG,ARM,AUS,AUT,AZE,BAH,...,URS,URU,USA,UZB,VEN,VIE,YUG,ZAM,ZIM,ZZX
1896,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,NaN,NaN,...,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0
1900,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,...,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0
1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,394.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0
1908,NaN,NaN,NaN,19.0,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912,NaN,NaN,NaN,10.0,NaN,NaN,NaN,14.0,NaN,NaN,...,NaN,NaN,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check if the transformation was correct

In [86]:
assert byCountry.iloc[0].sum() == medalists.groupby('Edition').get_group(1896).shape[0]
assert byCountry.loc[1912, 'USA'] == medalists.groupby(['Edition', 'NOC']).get_group((1912, 'USA')).shape[0]

As can be seen by the cell below, the number of medals awarded at each olympics changed over time. So let's transform the number of medals won by each country into the percentage of medals won by each country.

In [90]:
byCountry.sum(axis=1)

1896     151.0
1900     512.0
1904     470.0
1908     804.0
1912     885.0
1920    1298.0
1924     884.0
1928     710.0
1932     615.0
1936     875.0
1948     814.0
1952     889.0
1956     885.0
1960     882.0
1964    1010.0
1968    1031.0
1972    1185.0
1976    1305.0
1980    1387.0
1984    1459.0
1988    1546.0
1992    1705.0
1996    1859.0
2000    2015.0
2004    1998.0
2008    2042.0
dtype: float64

In [91]:
percByCountry = byCountry.apply(lambda x: x/x.sum(), axis=1)
percByCountry.head()

,AFG,AHO,ALG,ANZ,ARG,ARM,AUS,AUT,AZE,BAH,...,URS,URU,USA,UZB,VEN,VIE,YUG,ZAM,ZIM,ZZX
1896,NaN,NaN,NaN,NaN,NaN,NaN,0.013245,0.033113,NaN,NaN,...,NaN,NaN,0.132450,NaN,NaN,NaN,NaN,NaN,NaN,0.039735
1900,NaN,NaN,NaN,NaN,NaN,NaN,0.009766,0.011719,NaN,NaN,...,NaN,NaN,0.107422,NaN,NaN,NaN,NaN,NaN,NaN,0.066406
1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002128,NaN,NaN,...,NaN,NaN,0.838298,NaN,NaN,NaN,NaN,NaN,NaN,0.017021
1908,NaN,NaN,NaN,0.023632,NaN,NaN,NaN,0.001244,NaN,NaN,...,NaN,NaN,0.078358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912,NaN,NaN,NaN,0.011299,NaN,NaN,NaN,0.015819,NaN,NaN,...,NaN,NaN,0.114124,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check if it worked

In [92]:
assert percByCountry.loc[1896, 'USA'] ==  (byCountry.loc[1896, 'USA'] / byCountry.iloc[0].sum())

In [125]:
topSix= percByCountry.apply(np.mean, axis=0).sort_values(ascending=False).iloc[0:6]
countryCodes[countryCodes.IOC.isin(list(topSix.index))]

,Country,IOC,ISO
70,Germany,GER,DE
191,United States,USA,US


In [117]:
countryCodes.head()

,Country,IOC,ISO
0,Afghanistan,AFG,AF
1,Albania,ALB,AL
2,Algeria,ALG,DZ
3,American Samoa*,ASA,AS
4,Andorra,AND,AD


In [130]:
topSix
EUN
URS
GDR


URS    0.197122
USA    0.173034
EUN    0.130792
GDR    0.124298
EUA    0.092680
GER    0.087469
dtype: float64

In [131]:
byCountry.loc[:,'RUS']

1896      NaN
1900      NaN
1904      NaN
1908      NaN
1912      NaN
1920      NaN
1924      NaN
1928      NaN
1932      NaN
1936      NaN
1948      NaN
1952      NaN
1956      NaN
1960      NaN
1964      NaN
1968      NaN
1972      NaN
1976      NaN
1980      NaN
1984      NaN
1988      NaN
1992      NaN
1996    115.0
2000    188.0
2004    192.0
2008    143.0
Name: RUS, dtype: float64